In [19]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://metro.cdmx.gob.mx/longitud-de-estacion"
response = requests.get(url)
response.encoding = 'utf-8'  # Soluciona problemas de acentos/caracteres en origen
soup = BeautifulSoup(response.text, 'html.parser')

tabla = soup.find('table')
if not tabla:
    raise ValueError("Tabla no encontrada")

filas = tabla.find_all('tr')[1:]
linea_actual = None
datos = []

for fila in filas:
    celdas = fila.find_all('td')
    if not celdas:
        continue

    # Detectar nueva línea
    if celdas[0].get_text(strip=True).isdigit():
        linea_actual = celdas[0].get_text(strip=True)
        if len(celdas) >= 4:
            tramo = celdas[1].get_text(strip=True)
            longitud_estacion = celdas[2].get_text(strip=True)
            longitud_inter = celdas[3].get_text(strip=True)
            datos.append((linea_actual, tramo, longitud_estacion, longitud_inter))
    else:
        # Continuación de línea
        if len(celdas) >= 3 and linea_actual is not None:
            tramo = celdas[0].get_text(strip=True)
            longitud_estacion = celdas[1].get_text(strip=True)
            longitud_inter = celdas[2].get_text(strip=True)
            datos.append((linea_actual, tramo, longitud_estacion, longitud_inter))

# Crear DataFrame
df = pd.DataFrame(datos, columns=["Línea", "Tramo", "Longitud Estación", "Longitud Interestación"])

# Visualizar resultado
print(df.head())


  Línea                                   Tramo Longitud Estación  \
0     1                    Pantitlán - Zaragoza               150   
1     1                 Zaragoza - Gómez Farías               150   
2     1  Gómez Farías  - Boulevard Puerto Aéreo               150   
3     1       Boulevard Puerto Aéreo - Balbuena               150   
4     1                    Balbuena – Moctezuma               150   

  Longitud Interestación  
0                  1,320  
1                    762  
2                    611  
3                    595  
4                    703  


In [20]:
display(df.columns)
display(df.describe())

Index(['Línea', 'Tramo', 'Longitud Estación', 'Longitud Interestación'], dtype='object')

,Línea,Tramo,Longitud Estación,Longitud Interestación
count,195,195,195,195
unique,10,184,12,183
top,9,TOTALES,150,611
freq,43,12,181,2


In [21]:
import pandas as pd
import re

# 1. Si tienes el df cargado, filtramos basura inmediata
# Eliminamos filas donde 'Línea' sea 'TOTALES' o vacía
df = df[~df['Línea'].astype(str).str.contains("TOTALES", case=False, na=False)]

# 2. Renombrado inicial
df.rename(columns={
    "Línea": "linea", 
    "Longitud Interestación": "metros",
    "Tramo": "tramo_bruto"
}, inplace=True)

In [22]:
print("--- ANÁLISIS DE SEPARADORES ---")

# 1. Detectar guiones especiales (En-dash '–' y Em-dash '—')
# Usamos regex para buscar cualquiera de los dos caracteres
mask_raros = df['tramo_bruto'].astype(str).str.contains(r'[–—]', regex=True)
tramos_raros = df[mask_raros]

print(f"\n1. Tramos con guiones tipográficos (– / —): {len(tramos_raros)}")
if not tramos_raros.empty:
    print(tramos_raros[['linea', 'tramo_bruto']].to_string(index=False))

# 2. Detectar tramos que NO tienen el guion estándar (-)
# Esto revela si hay separadores desconocidos o filas basura
mask_sin_std = ~df['tramo_bruto'].astype(str).str.contains('-', regex=False)
# Excluimos los que ya detectamos en el paso 1 para ver errores "nuevos"
otros_errores = df[mask_sin_std & ~mask_raros]

--- ANÁLISIS DE SEPARADORES ---

1. Tramos con guiones tipográficos (– / —): 144
linea                                        tramo_bruto
    1                               Balbuena – Moctezuma
    1                            San Lázaro – Candelaria
    1                                Candelaria – Merced
    1                   Pino Suárez – Isabel la Católica
    1                Isabel la Católica – Salto del Agua
    1                          Salto del Agua – Balderas
    1                              Balderas – Cuauhtémoc
    1                           Cuauhtémoc – Insurgentes
    1                              Insurgentes – Sevilla
    1                              Sevilla – Chapultepec
    1                          Chapultepec – Juanacatlán
    1                             Juanacatlán – Tacubaya
    1                            Tacubaya – Observatorio
    2                         Cuatro Caminos – Panteones
    2                                 Panteones – Tacuba
    2  

In [ ]:
import pandas as pd
import re

print("--- INICIANDO LIMPIEZA Y CORRECCIÓN ---")

# 1. Renombrado Inicial
df.rename(columns={
    "Línea": "linea", 
    "Longitud Interestación": "metros",
    "Tramo": "tramo_bruto",
    "Longitud Estación": "col_desplazada" # Temporal para recuperar datos perdidos
}, inplace=True)

# 2. ELIMINAR BASURA (Filas TOTALES)
df = df[~df['tramo_bruto'].astype(str).str.contains("TOTALES", case=False, na=False)]

# 3. PARCHE DE DESPLAZAMIENTO (Corrección Líneas A y B)
# Detectamos donde el tramo es solo "A" o "B" y recuperamos la info de la columna siguiente

# Caso Línea A (Fila 144)
mask_a = df['tramo_bruto'].astype(str).str.strip() == 'A'
df.loc[mask_a, 'linea'] = 'A'
# Recuperamos el texto real que estaba en la columna incorrecta
df.loc[mask_a, 'tramo_bruto'] = df.loc[mask_a, 'col_desplazada'] 
# Corregimos metros manualmente (dato histórico CDMX) porque se perdió en el desplazamiento
df.loc[mask_a, 'metros'] = 1639 

# Caso Línea B (Fila 154)
mask_b = df['tramo_bruto'].astype(str).str.strip() == 'B'
df.loc[mask_b, 'linea'] = 'B'
df.loc[mask_b, 'tramo_bruto'] = df.loc[mask_b, 'col_desplazada']
df.loc[mask_b, 'metros'] = 574 # Dato histórico CDMX

# 4. NORMALIZACIÓN DE SEPARADORES (Tu código previo)
df['tramo_bruto'] = df['tramo_bruto'].astype(str) \
    .str.replace('–', '-', regex=False) \
    .str.replace('—', '-', regex=False) \
    .str.replace(r'\s*-\s*', '-', regex=True)

# 5. SEPARACIÓN ORIGEN / DESTINO
split_data = df['tramo_bruto'].str.split('-', expand=True, n=1)
df['origen'] = split_data[0].str.strip()
df['destino'] = split_data[1].str.strip()

# 6. LIMPIEZA DE METROS
# Aseguramos que sea string, quitamos comas y extraemos números
df['metros'] = df['metros'].astype(str).str.replace(',', '').str.extract(r'(\d+)').fillna(0).astype(int)

# 7. ASIGNAR ESTADO Y LIMPIEZA FINAL
df['estado'] = 1
df['linea'] = df['linea'].astype(str).str.strip()

# --- VERIFICACIÓN FINAL ---
errores = df[df['destino'].isnull()]
print(f"Filas procesadas: {len(df)}")
print(f"Errores restantes: {len(errores)}")

if not errores.empty:
    print("Filas con error:")
    print(errores[['linea', 'tramo_bruto']])
else:
    print("Tabla perfecta. Lista para subir.")
    # Mostrar corrección específica de A y B
    print(df[df['linea'].isin(['A', 'B'])].head(2)[['linea', 'origen', 'destino', 'metros']])

--- INICIANDO LIMPIEZA Y CORRECCIÓN ---
Filas procesadas: 183
Errores restantes: 0
✅ Tabla perfecta. Lista para subir.
    linea     origen            destino  metros
132     A  Pantitlán             Puebla    1380
144     A  Pantitlán  Agrícola Oriental    1639


In [ ]:
# 1. Crear la columna combinada "Tramo" (Origen - Destino)
df['tramo_formato'] = df['origen'] + ' - ' + df['destino']

# 2. Seleccionar columnas en el orden correcto para el parser de Kotlin
# Kotlin espera: partes[0]=Linea, partes[1]=Tramo, partes[2]=Metros, partes[3]=Estado
df_export = df[['linea', 'tramo_formato', 'metros', 'estado']]

# 3. Guardar a TXT con separador pipe (|)
nombre_archivo = "tramos_metro_final.txt"
df_export.to_csv(nombre_archivo, sep='|', index=False, encoding='utf-8')

print(f"Archivo generado: {nombre_archivo}")
print("--- PREVISUALIZACIÓN DEL CONTENIDO ---")

# 4. Leer las primeras 10 líneas para verificar visualmente
with open(nombre_archivo, "r", encoding="utf-8") as f:
    for _ in range(10):
        print(f.readline().strip())

✅ Archivo generado: tramos_metro_final.txt
--- PREVISUALIZACIÓN DEL CONTENIDO ---
linea|tramo_formato|metros|estado
1|Pantitlán - Zaragoza|1320|1
1|Zaragoza - Gómez Farías|762|1
1|Gómez Farías - Boulevard Puerto Aéreo|611|1
1|Boulevard Puerto Aéreo - Balbuena|595|1
1|Balbuena - Moctezuma|703|1
1|Moctezuma - San Lázaro|478|1
1|San Lázaro - Candelaria|866|1
1|Candelaria - Merced|698|1
1|Merced - Pino Suárez|745|1


In [27]:
# 1. Eliminar columna 'metros' del DataFrame actual
if 'metros' in df.columns:
    df = df.drop(columns=['metros'])

In [30]:
# 1. SELECCIÓN DE COLUMNAS NECESARIAS
# Descartamos 'tramo_bruto', 'col_desplazada' y 'tramo_formato'
df_final = df[['linea', 'origen', 'destino', 'estado']].copy()

print("--- DATOS LISTOS PARA SUBIR ---")
display(df_final)


--- DATOS LISTOS PARA SUBIR ---


,linea,origen,destino,estado
0,1,Pantitlán,Zaragoza,1
1,1,Zaragoza,Gómez Farías,1
2,1,Gómez Farías,Boulevard Puerto Aéreo,1
3,1,Boulevard Puerto Aéreo,Balbuena,1
4,1,Balbuena,Moctezuma,1
...,...,...,...,...
189,12,Eje Central,Parque de los Venados,1
190,12,Parque de los Venados,Zapata,1
191,12,Zapata,Hospital 20 de Noviembre,1
192,12,Hospital 20 de Noviembre,Insurgentes Sur,1


In [31]:
# ---------------------------------------------------------
# 2. CARGA A FIRESTORE (FINAL)
# ---------------------------------------------------------
import firebase_admin
from firebase_admin import credentials, firestore

def sanitize_id(raw):
    return raw.replace(" ", "_").replace("/", "_").replace(".", "_").replace("#", "_")

CREDENTIALS_PATH = "hackaton-ai-mobility-firebase-adminsdk-fbsvc-261bde2096.json"
def subir_datos_final():
    # Ajusta la ruta si es necesario
    if not firebase_admin._apps:
        cred = credentials.Certificate(CREDENTIALS_PATH) 
        firebase_admin.initialize_app(cred)
    
    db = firestore.client()
    batch = db.batch()
    count = 0
    total_tramos = 0
    estaciones_map = {} 

    print("\nIniciando carga a Firebase...")
    
    for _, row in df_final.iterrows():
        # ID: linea_origen_destino
        doc_id = sanitize_id(f"{row['linea']}_{row['origen']}_{row['destino']}")
        ref = db.collection('tramosBD').document(doc_id)
        
        # Datos a subir
        data = {
            "linea": str(row['linea']), # Asegurar string
            "origen": str(row['origen']),
            "destino": str(row['destino']),
            "estado": int(row['estado']) # Asegurar int
        }
        
        batch.set(ref, data)
        count += 1
        total_tramos += 1

        # Recolectar para Estaciones
        for est in [row['origen'], row['destino']]:
            key = est.lower()
            if key not in estaciones_map:
                estaciones_map[key] = {'nombre': est, 'lineas': set()}
            estaciones_map[key]['lineas'].add(str(row['linea']))

        if count >= 400:
            batch.commit()
            batch = db.batch()
            count = 0
            print(f"{total_tramos} tramos procesados...")

    if count > 0:
        batch.commit()

    print(f"\nTotal Tramos: {total_tramos}")
    print("--- PROCESANDO ESTACIONES ---")

    # Subida de Estaciones
    batch = db.batch()
    count = 0
    for _, info in estaciones_map.items():
        doc_id = sanitize_id(info['nombre'])
        ref = db.collection('estacionesBD').document(doc_id)
        
        data_est = {
            "nombre": info['nombre'],
            "lineas": sorted(list(info['lineas'])),
            "abierta": 1
        }
        batch.set(ref, data_est)
        count += 1
        
        if count >= 400:
            batch.commit()
            batch = db.batch()
            count = 0
            
    if count > 0:
        batch.commit()
    
    print("¡BASE DE DATOS ACTUALIZADA CORRECTAMENTE!")

# Ejecutar
subir_datos_final()


Iniciando carga a Firebase...

Total Tramos: 183
--- PROCESANDO ESTACIONES ---
¡BASE DE DATOS ACTUALIZADA CORRECTAMENTE!
